# IMPORTACION DE LIBRERIAS 

In [1]:
import pandas as pd
import numpy as np
import os
from nbconvert import HTMLExporter
import nbformat
import matplotlib.pyplot as plt
import calendar
from datetime import datetime, timedelta
import re
import warnings

# IMPORTANDO TABLA DE REDCAP

In [54]:
rut = 'TABLA_REDCAP.txt'
resultados_redcap = pd.read_csv(rut , sep = '\t' , encoding='ISO-8859-1')

# FUNCIONES PREVIAS PARA EL ALGORITMO DE CONTEO DE VOTOS

In [55]:
def Repartir_voto_AP(columna):
    matriz = np.array([[round(1/15,2), round(2/15,2), round(3/15,2), round(4/15,2), round(5/15,2)] ])
    columna -= 1
    return matriz[0, columna]

def Repartir_voto_Esp(fila,columna):
    matriz = np.array( [  [0, round(4/10,2), round(3/10,2), round(2/10,2), round(1/10,2)],
                          [round(1/7,2), 0, round(1/7,2), round(2/7,2), round(3/7,2)],
                          [round(2/6,2), round(1/6,2), 0, round(1/6,2), round(2/6,2)],
                          [round(3/7,2), round(2/7,2), round(1/7,2), 0, round(1/7,2)],
                          [round(1/10,2), round(2/10,2), round(3/10,2), round(4/10,2), 0]
                       ])
    fila -=1
    columna -= 1
    return matriz[fila, columna]

def calculando_matriz_repa_votos_3(data_frame,indice,matrix_especialistas_no_nan):
    escenario = matrix_especialistas_no_nan.size
    if escenario == 1:
        calculo_reparticiones = np.array([ 
                                   [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                   [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)], 
                                   [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)]
                                         ])
        suma_columnas = np.sum(calculo_reparticiones, axis=0)
        k = np.argmax(suma_columnas) + 1      
    elif escenario == 2:
        if data_frame.iloc[indice]['Especialista_1_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1 :
            buscar = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp = np.argmax(buscar) + 1
            calculo_reparticiones = np.array([ 
                                     [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                     [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)], 
                                     [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)],
                                     [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1          
        elif  data_frame.iloc[indice]['Especialista_2_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1:
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) + 1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']], 
                                [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)], 
                                [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)],
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
    elif escenario == 3: 
        k = 1
   return k

def calculando_matriz_repa_votos_4(data_frame,indice,matrix_especialistas_no_nan):
    escenario = matrix_especialistas_no_nan.size
    if escenario == 1: 
        calculo_reparticiones = np.array([ 
                                   [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                   [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)] 
                                         ])
        suma_columnas = np.sum(calculo_reparticiones, axis=0)
        k = np.argmax(suma_columnas) + 1 
    elif escenario == 2:
      if data_frame.iloc[indice]['Especialista_1_Total_k'] == 3 or data_frame.iloc[indice]['Especialista_2_Total_k'] == 3: 
        if data_frame.iloc[indice]['Especialista_1_Total_k'] == 3 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1 :
            buscar = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp = np.argmax(buscar) +1
            calculo_reparticiones = np.array([ 
                                     [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                     [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)], 
                                     [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1          
        elif  data_frame.iloc[indice]['Especialista_2_Total_k'] == 3 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1:
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']], 
                                [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1    
      elif data_frame.iloc[indice]['Especialista_1_Total_k'] == 2 or data_frame.iloc[indice]['Especialista_2_Total_k'] == 2: 
         k = 0
    elif escenario == 3:
        if data_frame.iloc[indice]['Especialista_1_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_3_Total_k'] == 1:            
            buscar = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']]])
            kp_1 = np.argmax(buscar_1) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
        elif data_frame.iloc[indice]['Especialista_2_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1  and data_frame.iloc[indice]['Especialista_3_Total_k'] == 1:             
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']]])
            kp_1 = np.argmax(buscar_1) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']], 
                                [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
        elif data_frame.iloc[indice]['Especialista_3_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1  and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1:             
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp_1 = np.argmax(buscar_1) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']], 
                                [Repartir_voto_AP(1), Repartir_voto_AP(2), Repartir_voto_AP(3), Repartir_voto_AP(4), Repartir_voto_AP(5)], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
    elif escenario == 4:
        k = 1
    return k  

def calculando_matriz_repa_votos_5(data_frame,indice,matrix_especialistas_no_nan):
    escenario = matrix_especialistas_no_nan.size
    if escenario == 1: 
      buscar_1 = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
      max_valor = np.max(buscar_1)
      if  np.sum(buscar_1 == max_valor) == 1 and np.sum(buscar_1 == 0) >= 1:
        k = np.argmax(buscar_1) + 1 
      if np.all(buscar_1==1):
        k = 3
      if np.sum(buscar_1 == 2) == 2 and np.sum(buscar_1 == 1) == 1 and np.sum(buscar_1 == 0) == 2:         
          f = np.where(buscar_1 == 1) [1][0] + 1 
          calculo_reparticiones = np.array([ [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']],
                                             [Repartir_voto_Esp(f,1),Repartir_voto_Esp(f,2),Repartir_voto_Esp(f,3),Repartir_voto_Esp(f,4),Repartir_voto_Esp(f,5)] 
                                                 ]) 
          suma_columnas = np.sum(calculo_reparticiones, axis=0)
          k = np.argmax(suma_columnas) + 1            
    elif escenario == 2: 
      if data_frame.iloc[indice]['Especialista_1_Total_k'] == 4 or data_frame.iloc[indice]['Especialista_2_Total_k'] == 4: 
          if data_frame.iloc[indice]['Especialista_1_Total_k'] == 4 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1 :
            buscar = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp = np.argmax(buscar) + 1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)]
                                ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
          elif data_frame.iloc[indice]['Especialista_2_Total_k'] == 4 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1 :             
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) + 1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)]
                                ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
      elif data_frame.iloc[indice]['Especialista_1_Total_k'] == 3 or data_frame.iloc[indice]['Especialista_2_Total_k'] == 3: 
        if data_frame.iloc[indice]['Especialista_1_Total_k'] == 3 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 2 :
            buscar = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp = np.argmax(buscar) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)]
                                ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) +1 
        elif data_frame.iloc[indice]['Especialista_2_Total_k'] == 3 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 2 :             
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)]
                                ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
    elif escenario == 3: 
      if data_frame.iloc[indice]['Especialista_1_Total_k'] == 3 or data_frame.iloc[indice]['Especialista_2_Total_k'] == 3 or data_frame.iloc[indice]['Especialista_3_Total_k'] == 3: 
        if data_frame.iloc[indice]['Especialista_1_Total_k'] == 3 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_3_Total_k'] == 1:
            buscar = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']]])
            kp_1 = np.argmax(buscar_1) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) +1 
        elif data_frame.iloc[indice]['Especialista_2_Total_k'] == 3 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_3_Total_k'] == 1:             
             buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']]])
            kp_1 = np.argmax(buscar_1) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
        elif data_frame.iloc[indice]['Especialista_3_Total_k'] == 3 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1:             
            buscar = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp_1 = np.argmax(buscar_1) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
      elif data_frame.iloc[indice]['Especialista_1_Total_k'] == 2 or data_frame.iloc[indice]['Especialista_2_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_3_Total_k'] == 2: 
        k = 0
    elif escenario == 4: 
      if data_frame.iloc[indice]['Especialista_1_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_3_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_4_Total_k'] == 1 :
            buscar = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp = np.argmax(buscar) + 1
            buscar_1 = np.array([[data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']]])
            kp_1 = np.argmax(buscar_1) + 1
            buscar_2 = np.array([[data_frame.iloc[indice]['D1'], data_frame.iloc[indice]['D2'], data_frame.iloc[indice]['D3'], data_frame.iloc[indice]['D4'], data_frame.iloc[indice]['D5']]])
            kp_2 = np.argmax(buscar_2) + 1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)],
                                [Repartir_voto_Esp(kp_2,1),Repartir_voto_Esp(kp_2,2),Repartir_voto_Esp(kp_2,3),Repartir_voto_Esp(kp_2,4),Repartir_voto_Esp(kp_2,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
      elif data_frame.iloc[indice]['Especialista_2_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_3_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_4_Total_k'] == 1:             
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']]])
            kp_1 = np.argmax(buscar_1) +1
            buscar_2 = np.array([[data_frame.iloc[indice]['D1'], data_frame.iloc[indice]['D2'], data_frame.iloc[indice]['D3'], data_frame.iloc[indice]['D4'], data_frame.iloc[indice]['D5']]])
            kp_2 = np.argmax(buscar_2) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)],
                                [Repartir_voto_Esp(kp_2,1),Repartir_voto_Esp(kp_2,2),Repartir_voto_Esp(kp_2,3),Repartir_voto_Esp(kp_2,4),Repartir_voto_Esp(kp_2,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
      elif data_frame.iloc[indice]['Especialista_3_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_4_Total_k'] == 1:             
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp_1 = np.argmax(buscar_1) +1
            buscar_2 = np.array([[data_frame.iloc[indice]['D1'], data_frame.iloc[indice]['D2'], data_frame.iloc[indice]['D3'], data_frame.iloc[indice]['D4'], data_frame.iloc[indice]['D5']]])
            kp_2 = np.argmax(buscar_2) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)],
                                [Repartir_voto_Esp(kp_2,1),Repartir_voto_Esp(kp_2,2),Repartir_voto_Esp(kp_2,3),Repartir_voto_Esp(kp_2,4),Repartir_voto_Esp(kp_2,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
      elif data_frame.iloc[indice]['Especialista_4_Total_k'] == 2 and data_frame.iloc[indice]['Especialista_1_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_2_Total_k'] == 1 and data_frame.iloc[indice]['Especialista_3_Total_k'] == 1:             
            buscar = np.array([[data_frame.iloc[indice]['A1'], data_frame.iloc[indice]['A2'], data_frame.iloc[indice]['A3'], data_frame.iloc[indice]['A4'], data_frame.iloc[indice]['A5']]])
            kp = np.argmax(buscar) +1
            buscar_1 = np.array([[data_frame.iloc[indice]['B1'], data_frame.iloc[indice]['B2'], data_frame.iloc[indice]['B3'], data_frame.iloc[indice]['B4'], data_frame.iloc[indice]['B5']]])
            kp_1 = np.argmax(buscar_1) +1
            buscar_2 = np.array([[data_frame.iloc[indice]['C1'], data_frame.iloc[indice]['C2'], data_frame.iloc[indice]['C3'], data_frame.iloc[indice]['C4'], data_frame.iloc[indice]['C5']]])
            kp_2 = np.argmax(buscar_2) +1
            calculo_reparticiones = np.array([ 
                                [data_frame.iloc[indice]['D1'], data_frame.iloc[indice]['D2'], data_frame.iloc[indice]['D3'], data_frame.iloc[indice]['D4'], data_frame.iloc[indice]['D5']], 
                                [Repartir_voto_Esp(kp,1),Repartir_voto_Esp(kp,2),Repartir_voto_Esp(kp,3),Repartir_voto_Esp(kp,4),Repartir_voto_Esp(kp,5)],
                                [Repartir_voto_Esp(kp_1,1),Repartir_voto_Esp(kp_1,2),Repartir_voto_Esp(kp_1,3),Repartir_voto_Esp(kp_1,4),Repartir_voto_Esp(kp_1,5)],
                                [Repartir_voto_Esp(kp_2,1),Repartir_voto_Esp(kp_2,2),Repartir_voto_Esp(kp_2,3),Repartir_voto_Esp(kp_2,4),Repartir_voto_Esp(kp_2,5)]
                                            ])
            suma_columnas = np.sum(calculo_reparticiones, axis=0)
            k = np.argmax(suma_columnas) + 1 
    elif escenario == 5: 
         k = 1
    return k     

# ALGORITMO DE CONTEO DE VOTOS

In [56]:
def algoritmo_automatizacion_votos (filosofias:str, resultados_redcap: pd.DataFrame):
 xy1 = 0
 xy2 = 0
 xy3 = 0
 w1 = 0
 w2 = 0
 w3 = 0
 x1 = 0
 x2 = 0
 x3 = 0
 x4 = 0
 x5 = 0
 z1 = 0
 z2 = 0
 z3 = 0
 z4 = 0
 z5 = 0
 z6 = 0
 z7 = 0
 filosofia = filosofias
 tabloide = resultados_redcap
 consolidado = {
   'GRUPO':[],
   'CIE-10':[],
   'ENFERMEDAD':[],
   'RESULTADO':[],
   'k atencion':[] }
 DF_CONSOLIDADO = pd.DataFrame(consolidado)
 for index, row in tabloide.iterrows():
     a = row['Grupo Etario']
     b = row['CIE-10']
     c = row['Enfermedad']
     matrix_especialistas = []
     total_especialistas = 0
     indice = index
     if row['Especialistas'] >= 3:         
         if row['Especialistas'] == 3:            
            matrix_especialistas = np.array([row['Especialista_1'], row['Especialista_2'], row['Especialista_3'], row['Especialista_4'], row['Especialista_5']])
            matrix_especialistas_no_nan = np.array([x for x in matrix_especialistas if pd.notna(x) and x is not None and x !='nan']) 
            total_especialistas = matrix_especialistas_no_nan.size
            if total_especialistas == 1: 
              w1 = w1+1
              especialista = row['Especialista_1']
              k = calculando_matriz_repa_votos_3(tabloide,indice,matrix_especialistas_no_nan)            
              fila_new = []
              fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k   } ]
              df_fila_new = pd.DataFrame(fila_new)
              DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)                                
            elif  total_especialistas == 2: 
              w2 = w2+1   
              if row['Especialista_1_Total_k'] == 2 and row['Especialista_2_Total_k'] == 1 :
                 especialista = row['Especialista_1'] 
              if row['Especialista_2_Total_k'] == 2 and row['Especialista_1_Total_k'] == 1 :
                 especialista = row['Especialista_2']            
              k = calculando_matriz_repa_votos_3(tabloide,indice,matrix_especialistas_no_nan)
              fila_new = []
              fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k   } ]
              df_fila_new = pd.DataFrame(fila_new)
              DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
            elif  total_especialistas == 3: 
               w3 = w3+1
               if a == 'A':
                  k = calculando_matriz_repa_votos_3(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'PEDIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'B':
                  k = calculando_matriz_repa_votos_3(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'PEDIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'C':
                  k = calculando_matriz_repa_votos_3(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'MEDICINA INTERNA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'D':
                  k = calculando_matriz_repa_votos_3(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'MEDICINA INTERNA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'E':
                  k = calculando_matriz_repa_votos_3(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'GERIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
         if row['Especialistas'] == 4:            
            matrix_especialistas = np.array([row['Especialista_1'], row['Especialista_2'], row['Especialista_3'], row['Especialista_4'], row['Especialista_5']])
            matrix_especialistas_no_nan = np.array([x for x in matrix_especialistas if pd.notna(x) and x is not None and x !='nan']) 
            total_especialistas = matrix_especialistas_no_nan.size
            if total_especialistas == 1:  
              especialista = row['Especialista_1']
              k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)            
              fila_new = []
              fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k   } ]
              df_fila_new = pd.DataFrame(fila_new)
              DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
              x1 = x1 + 1
            elif total_especialistas == 2: 
             if row['Especialista_1_Total_k'] == 3 or row['Especialista_2_Total_k'] == 3: 
               if row['Especialista_1_Total_k'] == 3 and row['Especialista_2_Total_k'] == 1 :
                especialista = row['Especialista_1']
               elif row['Especialista_2_Total_k'] == 3 and row['Especialista_1_Total_k'] == 1 :             
                especialista = row['Especialista_2']
               k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)            
               fila_new = []
               fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k} ]
               df_fila_new = pd.DataFrame(fila_new)
               DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               x2 = x2 + 1
             elif row['Especialista_1_Total_k'] == 2 or row['Especialista_2_Total_k'] == 2:            
                especialista = 'Criterio de Expertos'
                k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)            
                fila_new = []
                fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k   } ]
                df_fila_new = pd.DataFrame(fila_new)
                DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
                x3 = x3 + 1 
            elif total_especialistas == 3: 
               if row['Especialista_1_Total_k'] == 2 and row['Especialista_2_Total_k'] == 1 and row['Especialista_3_Total_k'] == 1:
                especialista = row['Especialista_1']
               elif row['Especialista_2_Total_k'] == 2 and row['Especialista_1_Total_k'] == 1  and row['Especialista_3_Total_k'] == 1:             
                especialista = row['Especialista_2']
               elif row['Especialista_3_Total_k'] == 2 and row['Especialista_1_Total_k'] == 1 and row['Especialista_2_Total_k'] == 1:             
                especialista = row['Especialista_3']
               k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)            
               fila_new = []
               fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k} ]
               df_fila_new = pd.DataFrame(fila_new)
               DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)   
               x4 = x4 + 1
            elif total_especialistas == 4: 
               x5=x5+1     
               if a == 'A':
                  k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'PEDIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'B':
                  k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'PEDIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'C':
                  k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'MEDICINA INTERNA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'D':
                  k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'MEDICINA INTERNA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'E':
                  k = calculando_matriz_repa_votos_4(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'GERIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
         if row['Especialistas'] == 5: 
            matrix_especialistas = np.array([row['Especialista_1'], row['Especialista_2'], row['Especialista_3'], row['Especialista_4'], row['Especialista_5']])
            matrix_especialistas_no_nan = np.array([x for x in matrix_especialistas if pd.notna(x) and x is not None and x !='nan']) 
            total_especialistas = matrix_especialistas_no_nan.size
            if total_especialistas == 1:  
              z1 = z1+1
              especialista = row['Especialista_1']
              k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)            
              fila_new = []
              fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k   } ]
              df_fila_new = pd.DataFrame(fila_new)
              DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
            elif total_especialistas == 2: 
             if row['Especialista_1_Total_k'] == 4 or row['Especialista_2_Total_k'] == 4: 
               if row['Especialista_1_Total_k'] == 4 and row['Especialista_2_Total_k'] == 1 :
                especialista = row['Especialista_1']
               elif row['Especialista_2_Total_k'] == 4 and row['Especialista_1_Total_k'] == 1 :             
                especialista = row['Especialista_2']
               k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)            
               fila_new = []
               fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k} ]
               df_fila_new = pd.DataFrame(fila_new)
               DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               z2 = z2 + 1
             elif row['Especialista_1_Total_k'] == 3 or row['Especialista_2_Total_k'] == 3: 
               if row['Especialista_1_Total_k'] == 3 and row['Especialista_2_Total_k'] == 2:
                especialista = row['Especialista_1']
               elif row['Especialista_2_Total_k'] == 3 and row['Especialista_1_Total_k'] == 2:             
                especialista = row['Especialista_2']
               k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)            
               fila_new = []
               fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k} ]
               df_fila_new = pd.DataFrame(fila_new)
               DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               z3 = z3 + 1
            elif total_especialistas == 3: 
             if row['Especialista_1_Total_k'] == 3 or row['Especialista_2_Total_k'] == 3 or row['Especialista_3_Total_k'] == 3: 
               if row['Especialista_1_Total_k'] == 3 and row['Especialista_2_Total_k'] == 1 and row['Especialista_3_Total_k'] == 1:
                especialista = row['Especialista_1']
               elif row['Especialista_2_Total_k'] == 3 and row['Especialista_1_Total_k'] == 1 and row['Especialista_3_Total_k'] == 1:             
                especialista = row['Especialista_2']
               elif row['Especialista_3_Total_k'] == 3 and row['Especialista_1_Total_k'] == 1 and row['Especialista_2_Total_k'] == 1:             
                especialista = row['Especialista_3']
               k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)            
               fila_new = []
               fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k} ]
               df_fila_new = pd.DataFrame(fila_new)
               DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               z4 = z4 + 1
             elif row['Especialista_1_Total_k'] == 2 or row['Especialista_2_Total_k'] == 2 and row['Especialista_3_Total_k'] == 2: 
               especialista = "Criterio de Expertos"
               k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)            
               fila_new = []
               fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k} ]
               df_fila_new = pd.DataFrame(fila_new)
               DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               z5 = z5 + 1
            elif total_especialistas == 4:  
               if row['Especialista_1_Total_k'] == 2 and row['Especialista_2_Total_k'] == 1 and row['Especialista_3_Total_k'] == 1 and row['Especialista_4_Total_k'] == 1 :
                especialista = row['Especialista_1']
               elif row['Especialista_2_Total_k'] == 2 and row['Especialista_1_Total_k'] == 1 and row['Especialista_3_Total_k'] == 1 and row['Especialista_4_Total_k'] == 1:             
                especialista = row['Especialista_2']
               elif row['Especialista_3_Total_k'] == 2 and row['Especialista_1_Total_k'] == 1 and row['Especialista_2_Total_k'] == 1 and row['Especialista_4_Total_k'] == 1:             
                especialista = row['Especialista_3']
               elif row['Especialista_4_Total_k'] == 2 and row['Especialista_1_Total_k'] == 1 and row['Especialista_2_Total_k'] == 1 and row['Especialista_3_Total_k'] == 1:             
                especialista = row['Especialista_4']
               k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)            
               fila_new = []
               fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': especialista ,'k atencion': k} ]
               df_fila_new = pd.DataFrame(fila_new)
               DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               z6 = z6 + 1
            elif total_especialistas == 5: 
               z7 = z7 +1
               if a == 'A':
                  k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'PEDIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'B':
                  k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'PEDIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'C':
                  k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'MEDICINA INTERNA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'D':
                  k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'MEDICINA INTERNA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
               elif a == 'E':
                  k = calculando_matriz_repa_votos_5(tabloide,indice,matrix_especialistas_no_nan)    
                  fila_new = []
                  fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': 'GERIATRIA' ,'k atencion': k   } ]
                  df_fila_new = pd.DataFrame(fila_new)
                  DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
     if row['Atencion_primaria'] >=3: 
       if row['Atencion_primaria'] == 3:
          xy1 = xy1 + 1 
          if row['MF'] == 3 and row['MG'] == 0 and row['MF o MG'] == 0:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)            
          if row['MF'] == 2 and row['MG'] == 1 and row['MF o MG'] == 0:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)  
          if row['MF'] == 2 and row['MG'] == 0 and row['MF o MG'] == 1:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 2 and row['MF o MG'] == 0:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 1 and row['MF o MG'] == 1:
             if filosofia == "C":
                resultado = "MF"
             elif filosofia == "A":
                resultado = "MG"          
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 0 and row['MF o MG'] == 2:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 3 and row['MF o MG'] == 0:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 2 and row['MF o MG'] == 1:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 1 and row['MF o MG'] == 2:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 0 and row['MF o MG'] == 3:
             if filosofia == "C":
                resultado = "MF"
             elif filosofia == "A":
                resultado = "MG"          
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
       elif row['Atencion_primaria'] == 4:
          xy2 = xy2 + 1 
          if row['MF'] == 4 and row['MG'] == 0 and row['MF o MG'] == 0:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)   
          if row['MF'] == 3 and row['MG'] == 1 and row['MF o MG'] == 0:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)   
          if row['MF'] == 3 and row['MG'] == 0 and row['MF o MG'] == 1:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)  
          if row['MF'] == 2 and row['MG'] == 2 and row['MF o MG'] == 0:
             if filosofia == "C":
                resultado = "MF"
             elif filosofia == "A":
                resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)             
          if row['MF'] == 2 and row['MG'] == 1 and row['MF o MG'] == 1:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)  
          if row['MF'] == 2 and row['MG'] == 0 and row['MF o MG'] == 2:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True) 
          if row['MF'] == 1 and row['MG'] == 3 and row['MF o MG'] == 0:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 2 and row['MF o MG'] == 1:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 1 and row['MF o MG'] == 2:
             if filosofia == "C":
                resultado = "MF"
             elif filosofia == "A":
                resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)   
          if row['MF'] == 1 and row['MG'] == 0 and row['MF o MG'] == 3:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 4 and row['MF o MG'] == 0:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 3 and row['MF o MG'] == 1:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 2 and row['MF o MG'] == 2:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 1 and row['MF o MG'] == 3:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 0 and row['MF o MG'] == 4:
             if filosofia == "C":
                resultado = "MF"
             elif filosofia == "A":
                resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True) 
       elif row['Atencion_primaria'] == 5:
          xy3 = xy3 + 1
          if row['MF'] == 5 and row['MG'] == 0 and row['MF o MG'] == 0:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)     
          if row['MF'] == 4 and row['MG'] == 1 and row['MF o MG'] == 0:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True) 
          if row['MF'] == 4 and row['MG'] == 0 and row['MF o MG'] == 1:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True) 
          if row['MF'] == 3 and row['MG'] == 2 and row['MF o MG'] == 0:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 3 and row['MG'] == 1 and row['MF o MG'] == 1:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 3 and row['MG'] == 0 and row['MF o MG'] == 2:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)         
          if row['MF'] == 2 and row['MG'] == 3 and row['MF o MG'] == 0:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 2 and row['MG'] == 2 and row['MF o MG'] == 1:
             if filosofia == "C":
                resultado = "MF"
             elif filosofia == "A":
                resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True) 
          if row['MF'] == 2 and row['MG'] == 1 and row['MF o MG'] == 2:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 2 and row['MG'] == 0 and row['MF o MG'] == 3:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 4 and row['MF o MG'] == 0:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 3 and row['MF o MG'] == 1:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 2 and row['MF o MG'] == 2:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 1 and row['MG'] == 1 and row['MF o MG'] == 3:
             if filosofia == "C":
                resultado = "MF"
             elif filosofia == "A":
                resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True) 
          if row['MF'] == 1 and row['MG'] == 0 and row['MF o MG'] == 4:
             resultado = "MF"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 5 and row['MF o MG'] == 0:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 4 and row['MF o MG'] == 1:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 3 and row['MF o MG'] == 2:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 2 and row['MF o MG'] == 3:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 1 and row['MF o MG'] == 4:
             resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
          if row['MF'] == 0 and row['MG'] == 0 and row['MF o MG'] == 5:
             if filosofia == "C":
                resultado = "MF"
             elif filosofia == "A":
                resultado = "MG"
             fila_new = []
             fila_new = [{'GRUPO':a , 'CIE-10':b ,'ENFERMEDAD':c , 'RESULTADO': resultado ,'k atencion': 0   } ]
             df_fila_new = pd.DataFrame(fila_new)
             DF_CONSOLIDADO = pd.concat([DF_CONSOLIDADO, df_fila_new], ignore_index=True)
 return DF_CONSOLIDADO   

# APLICANDO ALGORTIMO DE AUTOMATIZACION VOTOS

In [57]:
respuesta_votos_A = algoritmo_automatizacion_votos ('A', resultados_redcap)
respuesta_votos_C = algoritmo_automatizacion_votos ('C', resultados_redcap)

# COMPLETANDO CON EL CRITERIO DE PRINCIPIO AHORRO


In [59]:
condicion = (respuesta_votos_A['CIE-10'] == 'F80.8') & (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'H10.0') &  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'OFTALMOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'J30.0') &  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'J45.8') &  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'J98.6') &  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'K59.8') &  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'K59.9') &  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'L20.9')&  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'DERMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'L30.4') &  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'Z13.6') &  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'CARDIOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'D50.0') &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'E30.8') &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ENDOCRINOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'F84.4' ) &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PSIQUIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'F84.8') &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'NEUROLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'F90.0') &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PSIQUIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M25.4') &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M41.1') &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M54.1') &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'NEUROLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'N92.2') &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'GINECOLOGIA Y OBSTETRICIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'R63.0' ) &  (respuesta_votos_A['GRUPO'] == 'B')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PSIQUIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M54.1') &  (respuesta_votos_A['GRUPO'] == 'C')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'NEUROCIRUGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M65.3') &  (respuesta_votos_A['GRUPO'] == 'C')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 3

condicion = (respuesta_votos_A['CIE-10'] == 'M75.1') &  (respuesta_votos_A['GRUPO'] == 'C')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'MEDICINA FISICA Y REHABILITACION'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'T93.8') &  (respuesta_votos_A['GRUPO'] == 'C')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'MEDICINA FISICA Y REHABILITACION'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'Z54.4') &  (respuesta_votos_A['GRUPO'] == 'C')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'MEDICINA FISICA Y REHABILITACION'
respuesta_votos_A.loc[condicion, 'k atencion'] = 2

condicion = (respuesta_votos_A['CIE-10'] == 'D17.6')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'UROLOGIA GENERAL'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'G56.0')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'G56.2')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'I84.1')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'GASTROENTEROLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'I84.4')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'GASTROENTEROLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'I84.8')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'GASTROENTEROLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'K80.5')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'CIRUGIA GENERAL'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M17.5')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'MG'
respuesta_votos_A.loc[condicion, 'k atencion'] = 0

condicion = (respuesta_votos_A['CIE-10'] == 'M19.1')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M41.2')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M41.5')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M85.2')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'N20.0')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'UROLOGIA GENERAL'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion =(respuesta_votos_A['CIE-10'] == 'N81.6')&  (respuesta_votos_A['GRUPO'] == 'D')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'CIRUGIA GENERAL'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'E11.2')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'CIRUGIA GENERAL'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'E11.5')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'GERIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'I69.4')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'GERIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'K80.3')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'GASTROENTEROLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M16.1')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'GERIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M47.2')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'NEUROCIRUGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 2

condicion = (respuesta_votos_A['CIE-10'] == 'M51.2')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M51.8')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M54.1')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'NEUROLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 2

condicion = (respuesta_votos_A['CIE-10'] == 'M65.4')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_A['CIE-10'] == 'M75.3')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'MEDICINA FISICA Y REHABILITACION'
respuesta_votos_A.loc[condicion, 'k atencion'] = 2

condicion = (respuesta_votos_A['CIE-10'] == 'M75.4')&  (respuesta_votos_A['GRUPO'] == 'E')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'MG'
respuesta_votos_A.loc[condicion, 'k atencion'] = 0

condicion = (respuesta_votos_A['CIE-10'] == 'J30.3')&  (respuesta_votos_A['GRUPO'] == 'A')
respuesta_votos_A.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_A.loc[condicion, 'k atencion'] = 1

# COMPLETANDO CON EL CRITERIO DE PRINCIPIO CALIDAD


In [60]:
condicion = (respuesta_votos_C['CIE-10'] == 'F80.8') & (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'H10.0') &  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'OFTALMOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'J30.0') &  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'J45.8') &  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'J98.6') &  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'K59.8') &  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'K59.9') &  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'L20.9')&  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'DERMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'L30.4') &  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'Z13.6') &  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'CARDIOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'D50.0') &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'E30.8') &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ENDOCRINOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'F84.4' ) &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PSIQUIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'F84.8') &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'NEUROLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'F90.0') &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PSIQUIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M25.4') &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M41.1') &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M54.1') &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'NEUROLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'N92.2') &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'GINECOLOGIA Y OBSTETRICIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'R63.0' ) &  (respuesta_votos_C['GRUPO'] == 'B')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PSIQUIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M54.1') &  (respuesta_votos_C['GRUPO'] == 'C')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'NEUROCIRUGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M65.3') &  (respuesta_votos_C['GRUPO'] == 'C')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 3

condicion = (respuesta_votos_C['CIE-10'] == 'M75.1') &  (respuesta_votos_C['GRUPO'] == 'C')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'MEDICINA FISICA Y REHABILITACION'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'T93.8') &  (respuesta_votos_C['GRUPO'] == 'C')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'MEDICINA FISICA Y REHABILITACION'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'Z54.4') &  (respuesta_votos_C['GRUPO'] == 'C')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'MEDICINA FISICA Y REHABILITACION'
respuesta_votos_C.loc[condicion, 'k atencion'] = 2

condicion = (respuesta_votos_C['CIE-10'] == 'D17.6')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'UROLOGIA GENERAL'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'G56.0')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'G56.2')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'I84.1')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'GASTROENTEROLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'I84.4')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'GASTROENTEROLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'I84.8')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'GASTROENTEROLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'K80.5')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'CIRUGIA GENERAL'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M17.5')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'MF'
respuesta_votos_C.loc[condicion, 'k atencion'] = 0

condicion = (respuesta_votos_C['CIE-10'] == 'M19.1')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M41.2')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M41.5')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M85.2')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'N20.0')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'UROLOGIA GENERAL'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion =(respuesta_votos_C['CIE-10'] == 'N81.6')&  (respuesta_votos_C['GRUPO'] == 'D')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'CIRUGIA GENERAL'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'E11.2')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'CIRUGIA GENERAL'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'E11.5')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'GERIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'I69.4')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'GERIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'K80.3')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'GASTROENTEROLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M16.1')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'GERIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M47.2')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'NEUROCIRUGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 2

condicion = (respuesta_votos_C['CIE-10'] == 'M51.2')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M51.8')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M54.1')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'NEUROLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 2

condicion = (respuesta_votos_C['CIE-10'] == 'M65.4')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'ORTOPEDIA Y TRAUMATOLOGIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

condicion = (respuesta_votos_C['CIE-10'] == 'M75.3')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'MEDICINA FISICA Y REHABILITACION'
respuesta_votos_C.loc[condicion, 'k atencion'] = 2

condicion = (respuesta_votos_C['CIE-10'] == 'M75.4')&  (respuesta_votos_C['GRUPO'] == 'E')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'MF'
respuesta_votos_C.loc[condicion, 'k atencion'] = 0

condicion = (respuesta_votos_C['CIE-10'] == 'J30.3')&  (respuesta_votos_C['GRUPO'] == 'A')
respuesta_votos_C.loc[condicion, 'RESULTADO'] = 'PEDIATRIA'
respuesta_votos_C.loc[condicion, 'k atencion'] = 1

# EXPORTANDO LAS TABLAS DE CONSOLIDACION DE VOTOS

In [61]:
DF_CONSOLIDADO_respuesta_votos_C = respuesta_votos_C.rename(columns = {  'RESULTADO' : '_resultado' , 'k atencion' : '_k'})

DF_CONSOLIDADO_A_respuesta_votos_C = DF_CONSOLIDADO_respuesta_votos_C[DF_CONSOLIDADO_respuesta_votos_C['GRUPO'] == 'A']
DF_CONSOLIDADO_A_respuesta_votos_C = DF_CONSOLIDADO_A_respuesta_votos_C [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_A_respuesta_votos_C.to_csv('DF_CONSOLIDADO_A_respuesta_votos_C.txt', index=False , sep = '|')

DF_CONSOLIDADO_B_respuesta_votos_C = DF_CONSOLIDADO_respuesta_votos_C[DF_CONSOLIDADO_respuesta_votos_C['GRUPO'] == 'B']
DF_CONSOLIDADO_B_respuesta_votos_C = DF_CONSOLIDADO_B_respuesta_votos_C [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_B_respuesta_votos_C.to_csv('DF_CONSOLIDADO_B_respuesta_votos_C.txt', index=False , sep = '|')

DF_CONSOLIDADO_C_respuesta_votos_C = DF_CONSOLIDADO_respuesta_votos_C[DF_CONSOLIDADO_respuesta_votos_C['GRUPO'] == 'C']
DF_CONSOLIDADO_C_respuesta_votos_C = DF_CONSOLIDADO_C_respuesta_votos_C [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_C_respuesta_votos_C.to_csv('DF_CONSOLIDADO_C_respuesta_votos_C.txt', index=False , sep = '|')

DF_CONSOLIDADO_D_respuesta_votos_C = DF_CONSOLIDADO_respuesta_votos_C[DF_CONSOLIDADO_respuesta_votos_C['GRUPO'] == 'D']
DF_CONSOLIDADO_D_respuesta_votos_C = DF_CONSOLIDADO_D_respuesta_votos_C [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_D_respuesta_votos_C.to_csv('DF_CONSOLIDADO_D_respuesta_votos_C.txt', index=False , sep = '|')

DF_CONSOLIDADO_E_respuesta_votos_C = DF_CONSOLIDADO_respuesta_votos_C[DF_CONSOLIDADO_respuesta_votos_C['GRUPO'] == 'E']
DF_CONSOLIDADO_E_respuesta_votos_C = DF_CONSOLIDADO_E_respuesta_votos_C [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_E_respuesta_votos_C.to_csv('DF_CONSOLIDADO_E_respuesta_votos_C.txt', index=False , sep = '|')

DF_CONSOLIDADO_respuesta_votos_A = respuesta_votos_A.rename(columns = {  'RESULTADO' : '_resultado' , 'k atencion' : '_k'})

DF_CONSOLIDADO_A_respuesta_votos_A = DF_CONSOLIDADO_respuesta_votos_A[DF_CONSOLIDADO_respuesta_votos_A['GRUPO'] == 'A']
DF_CONSOLIDADO_A_respuesta_votos_A = DF_CONSOLIDADO_A_respuesta_votos_A [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_A_respuesta_votos_A.to_csv('DF_CONSOLIDADO_A_respuesta_votos_A.txt', index=False , sep = '|')

DF_CONSOLIDADO_B_respuesta_votos_A = DF_CONSOLIDADO_respuesta_votos_A[DF_CONSOLIDADO_respuesta_votos_A['GRUPO'] == 'B']
DF_CONSOLIDADO_B_respuesta_votos_A = DF_CONSOLIDADO_B_respuesta_votos_A [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_B_respuesta_votos_A.to_csv('DF_CONSOLIDADO_B_respuesta_votos_A.txt', index=False , sep = '|')

DF_CONSOLIDADO_C_respuesta_votos_A = DF_CONSOLIDADO_respuesta_votos_A[DF_CONSOLIDADO_respuesta_votos_A['GRUPO'] == 'C']
DF_CONSOLIDADO_C_respuesta_votos_A = DF_CONSOLIDADO_C_respuesta_votos_A [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_C_respuesta_votos_A.to_csv('DF_CONSOLIDADO_C_respuesta_votos_A.txt', index=False , sep = '|')

DF_CONSOLIDADO_D_respuesta_votos_A = DF_CONSOLIDADO_respuesta_votos_A[DF_CONSOLIDADO_respuesta_votos_A['GRUPO'] == 'D']
DF_CONSOLIDADO_D_respuesta_votos_A = DF_CONSOLIDADO_D_respuesta_votos_A [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_D_respuesta_votos_A.to_csv('DF_CONSOLIDADO_D_respuesta_votos_A.txt', index=False , sep = '|')

DF_CONSOLIDADO_E_respuesta_votos_A = DF_CONSOLIDADO_respuesta_votos_A[DF_CONSOLIDADO_respuesta_votos_A['GRUPO'] == 'E']
DF_CONSOLIDADO_E_respuesta_votos_A = DF_CONSOLIDADO_E_respuesta_votos_A [['CIE-10', '_resultado', '_k']]
DF_CONSOLIDADO_E_respuesta_votos_A.to_csv('DF_CONSOLIDADO_E_respuesta_votos_A.txt', index=False , sep = '|')

# PREPROCESAMIENTO DE DATOS DE LA TABLA DE ATENCIONES 

## IMPORTANDO TABLA DE ATENCIONES PERIODO 2022 - 2023

In [ ]:
ruta = r'atenciones2022_2023_id_encryt.csv'
Atenciones = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )

## RETIRANDO 'DIAGNOSTICOS' VACIOS

In [ ]:
datos_vacios = Atenciones['DIAGNOSTICO'].isna().sum()
Atenciones_1 = Atenciones.dropna(subset=['DIAGNOSTICO'])

## RETIRANDO SEXO = 2

In [64]:
Atenciones_1 = Atenciones_1[Atenciones_1['categoria'] !="otros"]
Atenciones_1 = Atenciones_1[Atenciones_1['sexo'] !=2]

## RETIRANDO SERVICIOS: MEDICO PERSONAL,MEDICO CONTROL , MEDICO OCUPACIONAL Y DEL MEDIO AMBIENTE

In [ ]:
Atenciones_1['SERVICIO'] = Atenciones_1['SERVICIO'].str.strip()
subcadenas = [
    'MEDICO DE PERSONAL',
    'MEDICINA OCUPACIONAL Y DEL MEDIO AMBIENTE',
    'MEDICO DE CONTROL \(MECO\)'
]
pattern = '|'.join(subcadenas)
Atenciones_1 = Atenciones_1[~Atenciones_1['SERVICIO'].str.contains(pattern, case=False, na=False)]

## HOMOGENEISANDO DESCRIPCION DE SERVICIOS

In [67]:
replacements = {
    'GINECOLOGIA': 'GINECOLOGIA Y OBSTETRICIA',
    'HEMATOLOGIA': 'HEMATOLOGIA CLINICA',
    'REANIMACION Y TERAPIA DEL DOLOR': 'ANESTESIA, ANALGESIA Y REANIMACION',
    'CIRUGIA VASCULAR': 'CIRUGIA DE TORAX Y CARDIOVASCULAR'
}
Atenciones_1['SERVICIO'] = Atenciones_1['SERVICIO'].replace(replacements)

In [71]:
# Grafico de atenciones mensuales por año

# Definir una función para mapear los valores de 'SERVICIO' a 'categoria_servicio'
def map_categoria_servicio(servicio):
    if servicio == 'MEDICINA GENERAL':
        return 'MG'
    elif servicio == 'MEDICINA FAMILIAR Y COMUNITARIA':
        return 'MF'
    else:
        return 'ESPECIALISTA'

# Aplicar la función a la columna 'SERVICIO' para crear la nueva columna 'categoria_servicio'
Atenciones_1['categoria_servicio'] = Atenciones_1['SERVICIO'].apply(map_categoria_servicio)


Atenciones_1_2022 = Atenciones_1[(Atenciones_1['FECHA_ATENCION'] < "01/01/2023") ]

Atenciones_1_2023 = Atenciones_1[Atenciones_1['FECHA_ATENCION'] >= "01/01/2023"]

## CONSOLIDANDO DATASET

In [74]:
Prime_Atenciones = Atenciones_1[['id','SERVICIO','DIAGNOSTICO','FECHA_ATENCION','categoria','anio','CENTRO']]

## DIVIENDO DATASET POR GRUPO ETARIO

In [ ]:
Prime_Atenciones_A = Prime_Atenciones[Prime_Atenciones['categoria'] == '1m a < 5a']
Prime_Atenciones_B = Prime_Atenciones[Prime_Atenciones['categoria'] == '5a a < 18a']
Prime_Atenciones_C = Prime_Atenciones[Prime_Atenciones['categoria'] == 'de 18a a < de 30a']
Prime_Atenciones_D = Prime_Atenciones[Prime_Atenciones['categoria'] == 'de 30a a < de 60a']
Prime_Atenciones_E = Prime_Atenciones[Prime_Atenciones['categoria'] == 'de 60a a mÃ¡s']

# ALGORITMO DE ASIGNACION DE ATENCIONES

In [ ]:
def algortimo_asignacion_atenciones(atenciones: pd.DataFrame, resultados: pd.DataFrame, numero_digitos_cie_10: int) -> pd.DataFrame:
    warnings.simplefilter(action='ignore', category=FutureWarning)
    atenciones['RESULTADO DE TIPO ATENCION'] = 'SIN ASIGNACION'
    atenciones['REVISADO'] = 'N'
    total_iteraciones = len(resultados)
    for i, fila in resultados.iterrows():
        a = fila[0]  
        b = fila[1]  
        c = fila[2]  
        print(f"Analizando la iteración {i} de {total_iteraciones} , el diagnostico {a}")
        if numero_digitos_cie_10 == 3:
            a_regex = re.escape(a[:3]) 
            Prime_Atenciones_G = atenciones.loc[atenciones['DIAGNOSTICO'].str.match(a_regex) & (atenciones['REVISADO'] == 'N')]
            Prime_Atenciones_2023 = Prime_Atenciones_G.loc[Prime_Atenciones_G['anio'] == 2023]
        else:
            Prime_Atenciones_G = atenciones[(atenciones['DIAGNOSTICO'] == a) & (atenciones['REVISADO'] == 'N')]
            Prime_Atenciones_2023 = Prime_Atenciones_G.loc[Prime_Atenciones_G['anio'] == 2023]
        if b == 'MF' or b == 'MG':
            indices = Prime_Atenciones_G.index
            atenciones.loc[indices, 'RESULTADO DE TIPO ATENCION'] = b
            atenciones.loc[indices, 'REVISADO'] = 'S'
        if b != 'MF' and b != 'MG' :
            if not Prime_Atenciones_2023.empty:
                Atenciones_id = Prime_Atenciones_2023.sort_values(by='FECHA_ATENCION', ascending=True)
                for _, filax in Atenciones_id.iterrows():
                    e_id = filax['id']
                    primer_fecha_atencion_2023 = filax['FECHA_ATENCION']
                    f_un_anio_atras = primer_fecha_atencion_2023 - timedelta(days=365)
                    indices = Prime_Atenciones_G[
                                    (Prime_Atenciones_G['id'] == e_id) &
                                    (Prime_Atenciones_G['FECHA_ATENCION'] >= f_un_anio_atras) & (Prime_Atenciones_G['REVISADO'] == 'N')
                                ].index                     
                    if c == 1:
                            atenciones.loc[indices, ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']
                    if c == 2:
                            atenciones.loc[indices[:1], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = ['MF', 'S']
                            atenciones.loc[indices[1:], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']
                    if c == 3:
                            atenciones.loc[indices[:2], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = ['MF', 'S']
                            atenciones.loc[indices[2:], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']
                    if c == 4:
                            atenciones.loc[indices[:3], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = ['MF', 'S']
                            atenciones.loc[indices[3:], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']
                    if c == 5:
                            atenciones.loc[indices[:4], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = ['MF', 'S']
                            atenciones.loc[indices[4:], ['RESULTADO DE TIPO ATENCION', 'REVISADO']] = [b, 'S']     
    warnings.simplefilter("default", FutureWarning)
    return atenciones

# APLICANDO ALGORTIMO DE AUTOMATIZACION VOTOS

## IMPORTANDO VOTOS CON FILOSOFIA CALIDAD

In [ ]:
# Grupo A
ruta = r'DF_CONSOLIDADO_A_respuesta_votos_C.txt'
datos_resultados_A_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_A_F_C = datos_resultados_A_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_A_F_C['_k'] = datos_resultados_A_F_C['_k'].astype(int)

# Grupo B
ruta = r'DF_CONSOLIDADO_B_respuesta_votos_C.txt'
datos_resultados_B_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_B_F_C = datos_resultados_B_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_B_F_C['_k'] = datos_resultados_B_F_C['_k'].astype(int)

# Grupo C
ruta = r'DF_CONSOLIDADO_C_respuesta_votos_C.txt'
datos_resultados_C_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_C_F_C = datos_resultados_C_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_C_F_C['_k'] = datos_resultados_C_F_C['_k'].astype(int)

# Grupo D
ruta = r'DF_CONSOLIDADO_D_respuesta_votos_C.txt'
datos_resultados_D_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_D_F_C = datos_resultados_D_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_D_F_C['_k'] = datos_resultados_D_F_C['_k'].astype(int)

# Grupo E
ruta = r'DF_CONSOLIDADO_E_respuesta_votos_C.txt'
datos_resultados_E_F_C = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_E_F_C = datos_resultados_E_F_C.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_E_F_C['_k'] = datos_resultados_E_F_C['_k'].astype(int)

## IMPORTANDO VOTOS CON FILOSOFIA AHORRRO

In [ ]:
# Grupo A
ruta = r'DF_CONSOLIDADO_A_respuesta_votos_A.txt'
datos_resultados_A_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_A_F_A = datos_resultados_A_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_A_F_A['_k'] = datos_resultados_A_F_A['_k'].astype(int)

# Grupo B
ruta = r'DF_CONSOLIDADO_B_respuesta_votos_A.txt'
datos_resultados_B_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_B_F_A = datos_resultados_B_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_B_F_A['_k'] = datos_resultados_B_F_A['_k'].astype(int)

# Grupo C
ruta = r'DF_CONSOLIDADO_C_respuesta_votos_A.txt'
datos_resultados_C_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_C_F_A = datos_resultados_C_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_C_F_A['_k'] = datos_resultados_C_F_A['_k'].astype(int)

# Grupo D
ruta = r'DF_CONSOLIDADO_D_respuesta_votos_A.txt'
datos_resultados_D_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_D_F_A = datos_resultados_D_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_D_F_A['_k'] = datos_resultados_D_F_A['_k'].astype(int)

# Grupo E
ruta = r'DF_CONSOLIDADO_E_respuesta_votos_A.txt'
datos_resultados_E_F_A = pd.read_csv(ruta , sep = '|' , encoding='ISO-8859-1' )
datos_resultados_E_F_A = datos_resultados_E_F_A.applymap(lambda x: x.strip() if isinstance(x, str) else x)
datos_resultados_E_F_A['_k'] = datos_resultados_E_F_A['_k'].astype(int)

## EJECUTANDO ALGORITMO

### ASIGNACION SEGUN RESULTADOS DE VOTOS PRINCIPIO CALIDAD / CIEI-10 CON 3 DIGITOS

In [ ]:
Resultado_A_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_A , datos_resultados_A_F_C,3)
Resultado_A_C_3D_Nuevo = Resultado_A_C_3D_Nuevo.sort_index()
Resultado_A_C_3D_Nuevo.to_csv('Resultado_A_C_3D_Nuevo.txt', index=False , sep = '|')

Resultado_B_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_B , datos_resultados_B_F_C,3)
Resultado_B_C_3D_Nuevo = Resultado_B_C_3D_Nuevo.sort_index()
Resultado_B_C_3D_Nuevo.to_csv('Resultado_B_C_3D_Nuevo.txt', index=False , sep = '|')

Resultado_C_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_C , datos_resultados_C_F_C,3)
Resultado_C_C_3D_Nuevo = Resultado_C_C_3D_Nuevo.sort_index()
Resultado_C_C_3D_Nuevo.to_csv('Resultado_C_C_3D_Nuevo.txt', index=False , sep = '|')

Resultado_D_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_D , datos_resultados_D_F_C,3)
Resultado_D_C_3D_Nuevo = Resultado_D_C_3D_Nuevo.sort_index()
Resultado_D_C_3D_Nuevo.to_csv('Resultado_D_C_3D_Nuevo.txt', index=False , sep = '|')

Resultado_E_C_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_E , datos_resultados_E_F_C,3)
Resultado_E_C_3D_Nuevo = Resultado_E_C_3D_Nuevo.sort_index()
Resultado_E_C_3D_Nuevo.to_csv('Resultado_E_C_3D_Nuevo.txt', index=False , sep = '|')

### ASIGNACION SEGUN RESULTADOS DE VOTOS PRINCIPIO AHORRO / CIEI-10 CON 3 DIGITOS

In [ ]:
Resultado_A_A_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_A , datos_resultados_A_F_A,3)
Resultado_A_A_3D_Nuevo = Resultado_A_A_3D_Nuevo.sort_index()
Resultado_A_A_3D_Nuevo.to_csv('Resultado_A_A_3D_Nuevo.txt', index=False , sep = '|')

Resultado_B_A_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_B , datos_resultados_B_F_A,3)
Resultado_B_A_3D_Nuevo = Resultado_B_A_3D_Nuevo.sort_index()
Resultado_B_A_3D_Nuevo.to_csv('Resultado_B_A_3D_Nuevo.txt', index=False , sep = '|')

Resultado_C_A_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_C , datos_resultados_C_F_A,3)
Resultado_C_A_3D_Nuevo = Resultado_C_A_3D_Nuevo.sort_index()
Resultado_C_A_3D_Nuevo.to_csv('Resultado_C_A_3D_Nuevo.txt', index=False , sep = '|')

Resultado_D_A_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_D , datos_resultados_D_F_A,3)
Resultado_D_A_3D_Nuevo = Resultado_D_A_3D_Nuevo.sort_index()
Resultado_D_A_3D_Nuevo.to_csv('Resultado_D_A_3D_Nuevo.txt', index=False , sep = '|')

Resultado_E_A_3D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_E , datos_resultados_E_F_A,3)
Resultado_E_A_3D_Nuevo = Resultado_E_A_3D_Nuevo.sort_index()
Resultado_E_A_3D_Nuevo.to_csv('Resultado_E_A_3D_Nuevo.txt', index=False , sep = '|')

### ASIGNACION SEGUN RESULTADOS DE VOTOS PRINCIPIO CALIDAD / CIEI-10 CON 4 DIGITOS

In [ ]:
Resultado_A_C_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_A , datos_resultados_A_F_C,4)
Resultado_A_C_4D_Nuevo = Resultado_A_C_4D_Nuevo.sort_index()
Resultado_A_C_4D_Nuevo.to_csv('Resultado_A_C_4D_Nuevo.txt', index=False , sep = '|')

Resultado_B_C_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_B , datos_resultados_B_F_C,4)
Resultado_B_C_4D_Nuevo = Resultado_B_C_4D_Nuevo.sort_index()
Resultado_B_C_4D_Nuevo.to_csv('Resultado_B_C_4D_Nuevo.txt', index=False , sep = '|')

Resultado_C_C_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_C , datos_resultados_C_F_C,4)
Resultado_C_C_4D_Nuevo = Resultado_C_C_4D_Nuevo.sort_index()
Resultado_C_C_4D_Nuevo.to_csv('Resultado_C_C_4D_Nuevo.txt', index=False , sep = '|')

Resultado_D_C_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_D , datos_resultados_D_F_C,4)
Resultado_D_C_4D_Nuevo = Resultado_D_C_4D_Nuevo.sort_index()
Resultado_D_C_4D_Nuevo.to_csv('Resultado_D_C_4D_Nuevo.txt', index=False , sep = '|')

Resultado_E_C_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_E , datos_resultados_E_F_C,4)
Resultado_E_C_4D_Nuevo = Resultado_E_C_4D_Nuevo.sort_index()
Resultado_E_C_4D_Nuevo.to_csv('Resultado_E_C_4D_Nuevo.txt', index=False , sep = '|')

### ASIGNACION SEGUN RESULTADOS DE VOTOS PRINCIPIO AHORRO / CIEI-10 CON 4 DIGITOS

In [ ]:
Resultado_A_A_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_A , datos_resultados_A_F_A,4)
Resultado_A_A_4D_Nuevo = Resultado_A_A_4D_Nuevo.sort_index()
Resultado_A_A_4D_Nuevo.to_csv('Resultado_A_A_4D_Nuevo.txt', index=False , sep = '|')

Resultado_B_A_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_B , datos_resultados_B_F_A,4)
Resultado_B_A_4D_Nuevo = Resultado_B_A_4D_Nuevo.sort_index()
Resultado_B_A_4D_Nuevo.to_csv('Resultado_B_A_4D_Nuevo.txt', index=False , sep = '|')

Resultado_C_A_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_C , datos_resultados_C_F_A,4)
Resultado_C_A_4D_Nuevo = Resultado_C_A_4D_Nuevo.sort_index()
Resultado_C_A_4D_Nuevo.to_csv('Resultado_C_A_4D_Nuevo.txt', index=False , sep = '|')

Resultado_D_A_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_D , datos_resultados_D_F_A,4)
Resultado_D_A_4D_Nuevo = Resultado_D_A_4D_Nuevo.sort_index()
Resultado_D_A_4D_Nuevo.to_csv('Resultado_D_A_4D_Nuevo.txt', index=False , sep = '|')

Resultado_E_A_4D_Nuevo = algortimo_asignacion_atenciones(Prime_Atenciones_E , datos_resultados_E_F_A,4)
Resultado_E_A_4D_Nuevo = Resultado_E_A_4D_Nuevo.sort_index()
Resultado_E_A_4D_Nuevo.to_csv('Resultado_E_A_4D_Nuevo.txt', index=False , sep = '|')